In [88]:
import json
import pandas as pd
import numpy as np
import cv2
import copy
import shutil
import os

In [118]:
def apply_color_mask(mask):
    original_height, original_width = mask.shape[:2]
    # Blue, Red, Green, Yellow, Orange, Purple
    color_order = [
        [255, 0, 0],
        [0, 0, 255],
        [0, 255, 0],
        [0, 255, 255],
        [0,165,255],
        [255,0,255]
    ]

    # Define the new dimensions (e.g., 50% of the original size)
    scale_percent = 40  # Percent of the original size
    new_width = int(original_width * scale_percent / 100)
    new_height = int(original_height * scale_percent / 100)
    new_dimensions = (new_width, new_height)

    # Resize the image
    mask = cv2.resize(mask, new_dimensions, interpolation = cv2.INTER_AREA)
    unique_instances = np.unique(mask)
    unique_classes = []

    # Output image initialized with zeros (black)
    colored_mask = np.zeros((*mask.shape, 3), dtype=np.uint8)

    for instance_value in unique_instances:
        # Extract instance ID
        instance_class = instance_value // 1000
        unique_classes.append(instance_class)
        instance_id = instance_value % 1000
        if instance_id <= 6:
            colored_mask[mask == instance_value] = color_order[instance_id - 1]
    print(np.unique(np.array(unique_classes)))
    return colored_mask


In [119]:
image_path = 'data/KS-FR-MONACO/29458/camcourt1_1512590601632_humans.png'
image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
result = apply_color_mask(image)
# Save the scaled mask
cv2.imwrite('color1.png', result)

[0 1]


True

In [120]:
image2 = 'data/KS-FR-MONACO/29458/camcourt2_1512590601632_humans.png'
image2 = cv2.imread(image2, cv2.IMREAD_UNCHANGED)
result2 = apply_color_mask(image2)
# Save the scaled mask
cv2.imwrite('color2.png', result2)

[0 1 2]


True

In [89]:
with open('data/player-positions.json', 'r') as f:
    player_data = json.load(f)

In [92]:
# given a mask object, return the bounding boxes for all player instances in order of instance ID
def get_bounding_boxes(mask, num_players):
    bounding_boxes = []
    max_instance_id = num_players
    img_height, img_width = mask.shape

    for instance_id in range(1, max_instance_id + 1):
        instance_value = 1000 + instance_id
        indices = np.where(mask == instance_value)
        min_x, min_y, max_x, max_y = float('inf'), float('inf'), float('-inf'), float('-inf')

        for y, x in zip(indices[0], indices[1]):
            if x < min_x:
                min_x = x
            if y < min_y:
                min_y = y
            if x > max_x:
                max_x = x
            if y > max_y:
                max_y = y
        if min_x != float('inf') and max_x != float('-inf'):
            width = max_x - min_x + 1
            height = max_y - min_y + 1
            center_x = min_x + width / 2
            center_y = min_y + height / 2
            normalized_center_x = center_x / img_width
            normalized_center_y = center_y / img_height
            normalized_width = width / img_width
            normalized_height = height / img_height
            bounding_boxes.append([normalized_center_x, normalized_center_y, normalized_width, normalized_height])
        else:
            bounding_boxes.append(None)
            print("Player mask not found")
    return bounding_boxes

In [93]:
clean_annotation_data = []
clean_position_data = []
for timestamp in player_data:
    num_players = len(timestamp['annotations'])
    if timestamp['side'] == 'left':
        camcourt = 'camcourt1_'
    else:
        camcourt = 'camcourt2_'
    image_heading = 'data/' + str(timestamp['arena_label']) + '/' + str(timestamp['game_id']) + '/'
    filename = camcourt + str(timestamp['timestamp']) + '_0.png'
    image_path = image_heading + filename
    mask_path = image_heading + filename[:-5] + 'humans.png'
    if timestamp['side'] == 'right':
        image = cv2.imread(image_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
        image = cv2.flip(image, 1)
        mask = cv2.flip(mask, 1)
        output_image_path = 'datasets/player-data/' + filename
        cv2.imwrite(output_image_path, image)
    else:
        mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
        shutil.copy(image_path, 'datasets/player-data/' + filename)
    boxes = get_bounding_boxes(mask, num_players)
    clean_annotation_data.append({
        'filename': filename,
        'bounding_boxes': boxes 
    })
    player_index = 0
    boxes_with_pos = []
    player_positions = []
    for annotation in timestamp['annotations']:
        boxes_with_pos.append(boxes[player_index])
        player_positions.append(annotation['xyz'])
        player_index += 1
    clean_position_data.append({
        'filename': filename,
        'filepath': image_path,
        'bounding_boxes': boxes_with_pos,
        'xyz_positions': player_positions
    })

clean_annotation_data = pd.DataFrame(clean_annotation_data)
clean_position_data = pd.DataFrame(clean_position_data)
def find_num_missing(boxes):
    none_count = 0
    for box in boxes:
        if box == None:
            none_count += 1
    return none_count
clean_annotation_data['num_missing'] = clean_annotation_data['bounding_boxes'].apply(find_num_missing)

In [99]:
clean_position_data.head()

,filename,filepath,bounding_boxes,xyz_positions
0,camcourt1_1512760174441_0.png,data/KS-FR-STCHAMOND/24320/camcourt1_151276017...,"[[0.6591748768472906, 0.37763371150729336, 0.0...","[[694.2688390833725, 65.61677260331226, -180.0]]"
1,camcourt2_1512760174441_0.png,data/KS-FR-STCHAMOND/24320/camcourt2_151276017...,"[[0.48460591133004927, 0.40640194489465153, 0....","[[362.794699732086, 579.1278461732304, -180.0]..."
2,camcourt2_1512761974116_0.png,data/KS-FR-STCHAMOND/24320/camcourt2_151276197...,"[[0.6268472906403941, 0.5587520259319287, 0.03...","[[1012.0560813562383, 1284.0197861083373, -180..."
3,camcourt1_1512763413947_0.png,data/KS-FR-STCHAMOND/24320/camcourt1_151276341...,"[[0.8103448275862069, 0.5162074554294975, 0.04...","[[1226.4846904387414, 921.1166040980411, -180...."
4,camcourt2_1512763413947_0.png,data/KS-FR-STCHAMOND/24320/camcourt2_151276341...,"[[0.9818349753694581, 0.5421393841166937, 0.03...","[[773.0160440625336, 53.63298153323194, -180.0..."


In [106]:
clean_position_data.to_csv('player_positions.csv', index=False)

In [122]:
clean_annotation_data['is_training'] = np.concatenate([np.array([True] * int(0.8 * len(clean_annotation_data))), np.array([False] * (len(clean_annotation_data) - int(0.8 * len(clean_annotation_data))))])
clean_annotation_data['is_training'] = clean_annotation_data['is_training'].sample(frac=1).reset_index(drop=True)

In [131]:
clean_annotation_data.head()

,filename,bounding_boxes,num_missing,is_training
0,camcourt1_1512760174441_0.png,"[[0.6591748768472906, 0.37763371150729336, 0.0...",0,True
1,camcourt2_1512760174441_0.png,"[[0.48460591133004927, 0.40640194489465153, 0....",0,True
2,camcourt2_1512761974116_0.png,"[[0.6268472906403941, 0.5587520259319287, 0.03...",0,True
3,camcourt1_1512763413947_0.png,"[[0.8103448275862069, 0.5162074554294975, 0.04...",0,True
4,camcourt2_1512763413947_0.png,"[[0.9818349753694581, 0.5421393841166937, 0.03...",0,True


In [123]:
for index, row in clean_annotation_data.iterrows():
    filename = row['filename']
    label_filename = filename[:-4] + '.txt'
    bounding_boxes = row['bounding_boxes']
    if row['is_training']:
        label_filepath = os.path.join('datasets/player-data/train/labels', label_filename)
        shutil.move('datasets/player-data/' + filename, 'datasets/player-data/train/images')
    else:
        label_filepath = os.path.join('datasets/player-data/val/labels', label_filename)
        shutil.move('datasets/player-data/' + filename, 'datasets/player-data/val/images')
    with open(label_filepath, 'w') as f:
        for box in bounding_boxes:
            f.write(f"0 {box[0]} {box[1]} {box[2]} {box[3]}\n")